custom layered cnn implementation trial
notes: 
- keras class_weight ?

In [2]:
import os
import joblib
import numpy as np
import cv2
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import (
    Input, Conv2D, MaxPooling2D, BatchNormalization, 
    Dropout, Flatten, Dense, LeakyReLU, ReLU
)
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, train_test_split
from sklearn.base import ClassifierMixin
from sklearn.metrics import f1_score, accuracy_score, classification_report
import os
import cv2
import numpy as np
from tqdm import tqdm

class SklearnKerasClassifier(KerasClassifier, ClassifierMixin):
    def __init__(self, model=None, **kwargs):
        super().__init__(model=model, **kwargs)
    
    @property
    def _tags(self):
        return self.model._tags if hasattr(self.model, "_tags") else {"binary_only": True}

def create_custom_cnn(
    input_shape=None,
    conv_blocks=((32, (3,3)), (64, (3,3))),
    dense_layers=(128,),
    dropout_rate=0.4,
    activation='relu',
    meta=None
):
    """Conv -> BN -> Activation -> Pool -> Dropout"""
    if input_shape is None:
        if meta is None: raise ValueError("meta veya input_shape parametresi yok")
        input_shape = meta["X_shape_"][1:]

    model = Sequential(name="Custom_CNN")
    model.add(Input(shape=input_shape))

    for filters, kernel_size in conv_blocks:
        model.add(Conv2D(filters, kernel_size, padding='same'))
        model.add(BatchNormalization())
        if activation == 'leaky_relu':
            model.add(LeakyReLU(alpha=0.1))
        else:
            model.add(ReLU())
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(dropout_rate))
    model.add(Flatten())
    
    for units in dense_layers:
        model.add(Dense(units))
        model.add(BatchNormalization())
        if activation == 'leaky_relu':
            model.add(LeakyReLU(alpha=0.1))
        else:
            model.add(ReLU())
        model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))

    return model

def is_dfire_image_fire(annotation_path, fire_class_ids):
    if not os.path.exists(annotation_path): return False
    try:
        with open(annotation_path, 'r') as f:
            lines = f.readlines()
        for line in lines:
            parts = line.strip().split(' ')
            if parts:
                class_id = int(parts[0])
                if class_id in fire_class_ids: return True
    except (ValueError, IOError): pass
    return False

def load_prep_4_cnn(data_dir, target_size=(128, 128)):
    all_images = []
    all_labels = []
    images_dir = os.path.join(data_dir, 'images')
    labels_dir = os.path.join(data_dir, 'labels')
    if not os.path.isdir(images_dir): return np.array([]), np.array([])
    if not os.path.isdir(labels_dir): return np.array([]), np.array([])

    img_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.gif')
    annotation_extension = '.txt'
    fire_class_ids = [0, 1]
    image_files = [f for f in os.listdir(images_dir) if f.lower().endswith(img_extensions)]
    if not image_files: return np.array([]), np.array([])
    for img_name in tqdm(image_files, desc="d-fire prep:"):
        img_path = os.path.join(images_dir, img_name)
        img_name_without_ext = os.path.splitext(img_name)[0]
        annotation_path = os.path.join(labels_dir, img_name_without_ext + annotation_extension)
        label = 1 if is_dfire_image_fire(annotation_path, fire_class_ids) else 0
        try:
            img = cv2.imread(img_path)
            if img is None: continue
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img_resized = cv2.resize(img, target_size, interpolation=cv2.INTER_LINEAR)
            img_normalized = img_resized.astype(np.float32) / 255.0
            all_images.append(img_normalized)
            all_labels.append(label)
        except Exception as e: continue
    return np.array(all_images), np.array(all_labels)

if __name__ == "__main__":
    data_directory = os.path.join('..', 'data_subsets', 'D-Fire', 'train')
    target_image_width = 128
    target_image_height = 128
    
    X, y = load_prep_4_cnn(data_directory, target_size=(target_image_width, target_image_height))

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
    print(f"train shape: {X_train.shape} - test shape: {X_test.shape}")

    keras_cnn_estimator = SklearnKerasClassifier(
        model=create_custom_cnn,
        loss='binary_crossentropy',
        optimizer=tf.keras.optimizers.Adam,
        optimizer__learning_rate=0.001,
        epochs=10,
        batch_size=16,
        verbose=0,
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=0, restore_best_weights=True)]
    )

    param_grid = {
        'model__conv_blocks': [
            ((16, (3,3)),),   ((32, (3,3)),),                         
            ((16, (5,5)), (32, (3,3))), ((32, (3,3)), (64, (3,3))), ((64, (5,5)), (128, (3,3))),  ((32, (5,5)), (64, (3,3))),
            ((32, (3,3)), (64, (3,3)), (128, (3,3))),
        ],
        'model__dense_layers': [(64,), (128,), (128, 64), (256,128),],
        'model__dropout_rate': [0.3, 0.4, 0.5],
        'model__activation': ['relu', 'leaky_relu'],
        'optimizer__learning_rate': [0.001, 0.0005, 0.0002, 0.0001]
    }

    cv_strategy = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)
    search_cv = RandomizedSearchCV(
        estimator=keras_cnn_estimator,
        param_distributions=param_grid,
        n_iter=8,
        cv=cv_strategy,
        scoring='f1',
        verbose=2,
        random_state=42
    )

    try:
        search_cv.fit(X_train, y_train, validation_split=0.2)
        print("\n--- hp opt results: ---")
        print(f"F1 (CV): {search_cv.best_score_:.4f}")
        print("best_params_:")
        print(search_cv.best_params_)
        
        best_model = search_cv.best_estimator_
        y_pred = best_model.predict(X_test)
        
        print("\n--- opt test results: ---")
        print(f"accuracy: {accuracy_score(y_test, y_pred):.4f}")
        print(f"F1: {f1_score(y_test, y_pred):.4f}")
        print("\nclass report:")
        print(classification_report(y_test, y_pred))

    except Exception as e:
        print(f"\n{e}")
        import traceback
        traceback.print_exc()

d-fire prep::   0%|          | 0/1629 [00:00<?, ?it/s]

d-fire prep:: 100%|██████████| 1629/1629 [00:38<00:00, 42.61it/s] 


train shape: (1221, 128, 128, 3) - test shape: (408, 128, 128, 3)
Fitting 4 folds for each of 8 candidates, totalling 32 fits


C:\Users\BerenÜnveren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


[CV] END model__activation=leaky_relu, model__conv_blocks=((16, (3, 3)),), model__dense_layers=(128, 64), model__dropout_rate=0.3, optimizer__learning_rate=0.0005; total time=  57.2s


...

[CV] END model__activation=leaky_relu, model__conv_blocks=((16, (3, 3)),), model__dense_layers=(128, 64), model__dropout_rate=0.3, optimizer__learning_rate=0.0002; total time=  52.3s


C:\Users\BerenÜnveren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


[CV] END model__activation=leaky_relu, model__conv_blocks=((16, (3, 3)),), model__dense_layers=(128, 64), model__dropout_rate=0.3, optimizer__learning_rate=0.0002; total time=  32.1s
[CV] END model__activation=relu, model__conv_blocks=((32, (3, 3)), (64, (3, 3)), (128, (3, 3))), model__dense_layers=(128,), model__dropout_rate=0.5, optimizer__learning_rate=0.0002; total time= 1.2min
[CV] END model__activation=relu, model__conv_blocks=((32, (3, 3)), (64, (3, 3)), (128, (3, 3))), model__dense_layers=(128,), model__dropout_rate=0.5, optimizer__learning_rate=0.0002; total time= 1.1min
[CV] END model__activation=relu, model__conv_blocks=((32, (3, 3)), (64, (3, 3)), (128, (3, 3))), model__dense_layers=(128,), model__dropout_rate=0.5, optimizer__learning_rate=0.0002; total time= 1.0min
[CV] END model__activation=relu, model__conv_blocks=((32, (3, 3)), (64, (3, 3)), (128, (3, 3))), model__dense_layers=(128,), model__dropout_rate=0.5, optimizer__learning_rate=0.0002; total time= 1.0min
[CV] END 

C:\Users\BerenÜnveren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(



--- hp opt results: ---
F1 (CV): 0.7201
best_params_:
{'optimizer__learning_rate': 0.0002, 'model__dropout_rate': 0.3, 'model__dense_layers': (128, 64), 'model__conv_blocks': ((16, (3, 3)),), 'model__activation': 'leaky_relu'}

--- opt test results: ---
accuracy: 0.7475
F1: 0.7268

class report:
              precision    recall  f1-score   support

           0       0.73      0.81      0.77       208
           1       0.77      0.69      0.73       200

    accuracy                           0.75       408
   macro avg       0.75      0.75      0.75       408
weighted avg       0.75      0.75      0.75       408

